# Encontrar audios duplicados en videos de television

Extract audio

In [ ]:
import os
import subprocess

from audio_extractor import extract_audio
from auxilar_functions import get_video_names, get_video_and_audio_paths

work_path = "C:/Users/iggym/Documents/Recuperacion de Informacion Multimedia/rim_project/movies/"
datasets = ["a", "b", "c"]

video_names = get_video_names(datasets, work_path)

video_paths, audio_paths = get_video_and_audio_paths(work_path, datasets, video_names)

sample_rate = 22000

for audio_path in audio_paths:
    if not os.path.isdir(audio_path):
        os.mkdir(audio_path)

j = 0
for i in range(len(datasets)):
    for video_name in video_names[i]:
        extract_audio(video_paths[j], sample_rate, f"{audio_paths[i]}/{video_name}")
        j+=1

Generate descriptors

In [ ]:
from audio_descriptor_generator import run_process
from auxilar_functions import get_video_names, get_video_and_audio_paths

work_path = "C:/Users/iggym/Documents/Recuperacion de Informacion Multimedia/rim_project/movies/"
datasets = ["a", "b", "c"]
sample_rate = 22000

video_names = get_video_names(datasets, work_path)
video_paths, audio_paths = get_video_and_audio_paths(work_path, datasets, video_names)

descriptor_paths = []
for dataset in datasets:
    descriptor_paths.append(f"{work_path}dataset_{dataset}/descriptors")

for descriptor_path in descriptor_paths:
    if not os.path.isdir(descriptor_path):
        os.mkdir(descriptor_path)

j = 0
for i in range(len(datasets)):
    for video_name in video_names[i]:
        print(f"    Progress: {j+1}/{len(video_paths)}")
        run_process(f"{audio_paths[i]}/{video_name}.{sample_rate}.wav", f"{video_name}_", f"{work_path}dataset_{datasets[i]}/")
        j+=1

Get results

In [ ]:
from main import run_process, show_results

work_path = "C:/Users/iggym/Documents/Recuperacion de Informacion Multimedia/rim_project/movies/"
datasets = ["a", "b", "c"]

video_names = get_video_names(datasets, work_path)

sequence_length = [3, 10, 30]
number_of_neighbours = [5, 10, 20]

descriptors_per_second = 2
descriptors_shapes = [[7177, 7201], [7201, 7202], [7023, 7201]]

descriptor_files = []

for j in range(len(datasets)):
    dataset = datasets[j]
    for i in range(len(video_names[j])):
        descriptor_files.append(f"{work_path}dataset_{dataset}/descriptors/{video_names[j][i]}__{descriptors_per_second}_{descriptors_shapes[j][i]}.bin")

actual_results = []
progress = 0
total_progress = len(datasets) * len(sequence_length) * len(number_of_neighbours)
for i in range(len(datasets)):
    for seq_len in sequence_length:
        for num_neighs in number_of_neighbours:
            progress += 1
            print(f"Dataset {datasets[i]} | Seq Len {seq_len} | Num Neighs {num_neighs} | Progress {progress}/{total_progress}")
            results, descriptors_per_second = run_process(descriptor_files[i*2:i*2+2], seq_len, num_neighs, same_audio=False)
            actual_results.append(results)

Evaluate results

In [ ]:
import pandas
from auxilar_functions import read_target_matrix, get_evaluations

all_evaluations = []

results_index = 0
for i in range(len(datasets)):
    dataset = datasets[i]
    target_file = f"{work_path}dataset_{dataset}/gt.txt"
    
    for j in range(len(sequence_length)):
        for w in range(len(number_of_neighbours)):
            print(f"Progress {round(results_index/len(actual_results)*100, 2)}%", end="\r")
            target_matrix = read_target_matrix(target_file)
            results = actual_results[results_index]
            evaluation = get_evaluations(target_matrix, results, video_names[i])
            all_evaluations.append([dataset, sequence_length[j], number_of_neighbours[w], evaluation])
            results_index += 1

Calculate Precision

In [ ]:
new_stats = []
for i in range(len(all_evaluations)):
    results = actual_results[i]
    total_results = len(results)
    df = pandas.DataFrame(all_evaluations[i][3], columns=["Match", "Comercial", "matches?"])
    false_positives = len(df[df["matches?"] == 0])
    true_positives = len(df[df["matches?"] == 1])
    if true_positives + false_positives == 0:
        precision = 0
    else:
        precision = round(true_positives / (true_positives + false_positives) * 100, 2)
    stats = all_evaluations[i][:3]
    stats.append(precision)
    new_stats.append(stats)

headers = ["Dataset", "Sequence Length", "Number of Neighbours", "Precision"]

final_stats = pandas.DataFrame(new_stats, columns=headers)


## Plots

Dataset A

In [ ]:
df1 = final_stats[final_stats["Dataset"] == "a"].pivot(index="Sequence Length", columns="Number of Neighbours", values="Precision")
df2 = final_stats[final_stats["Dataset"] == "b"].pivot(index="Sequence Length", columns="Number of Neighbours", values="Precision")
df3 = final_stats[final_stats["Dataset"] == "c"].pivot(index="Sequence Length", columns="Number of Neighbours", values="Precision")

df1.style.background_gradient(cmap="Blues")

Dataset B

In [ ]:
df2.style.background_gradient(cmap="Reds")

Dataset C

In [ ]:
df3.style.background_gradient(cmap="Greens")